In [26]:
import os
from langchain_ollama.llms import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [5]:
curr_dir=os.getcwd()
db_dir=os.path.join(curr_dir,'db')
persistent_directory=os.path.join(db_dir,'chroma_db_koa')

In [6]:
model=OllamaLLM(model='gemma3:4b')
embedding=OllamaEmbeddings(model='nomic-embed-text')

In [7]:
def create_vector_embedding(db_dir,persistent_directory):
    if not os.path.exists(persistent_directory):
        text_files=[f for f in os.listdir(db_dir) if f.endswith('.txt')]
        all_loaded_documents=[]
        for files in text_files:
            file_path=os.path.join(db_dir,files)
            loader=TextLoader(file_path)
            docs=loader.load()
            for doc in docs:
                doc.metadata={'source': files}
                all_loaded_documents.append(doc)
            print(f'Loaded {len(docs)} documents from {files}')
        text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
        docs=text_splitter.split_documents(all_loaded_documents)
        print('-----Document chunk information-----\n')
        print(f'Number of documnet chunks: {len(docs)}')
        print(f"Creating and persisting Chroma DB to '{persistent_directory}'...")
        try:
            db=Chroma.from_documents(documents=docs,embedding=embedding,persist_directory=persistent_directory)
            print('Successfully created and stored vector embedding')
        except Exception as e:
            print(f'Error while creating vector store: {e}')
    else:
        print('Vector embedding is already exists')


In [8]:
create_vector_embedding(db_dir,persistent_directory)

Loaded 1 documents from diagnosis.txt
Loaded 1 documents from faq.txt
Loaded 1 documents from grading.txt
Loaded 1 documents from surgery.txt
Loaded 1 documents from symptoms.txt
Loaded 1 documents from treatment.txt
-----Document chunk information-----

Number of documnet chunks: 49
Creating and persisting Chroma DB to 'e:\Learning Zone\DL\Mini project\db\chroma_db_koa'...
Successfully created and stored vector embedding


In [9]:
db=Chroma(persist_directory=persistent_directory,embedding_function=embedding)

C:\Users\Pradeesh\AppData\Local\Temp\ipykernel_19872\3314761616.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db=Chroma(persist_directory=persistent_directory,embedding_function=embedding)


In [23]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [25]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key='answer'
)

C:\Users\Pradeesh\AppData\Local\Temp\ipykernel_19872\860855868.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [27]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
If the follow up question is already a standalone question or if the chat history is empty, DO NOT rephrase the question and just return the original follow up question.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [28]:
QA_PROMPT_TEMPLATE = """You are a helpful AI assistant specializing in Knee Osteoarthritis (KOA).
Your goal is to answer questions about KOA based *only* on the provided context.
If the context does not contain the answer to the question, state clearly that you cannot answer based on the provided information.
Do not make up information or answer from your general knowledge if it's not in the context.
Be concise and informative.

Context:
{context}

Question: {question}

Helpful Answer:"""
QA_PROMPT = ChatPromptTemplate.from_template(QA_PROMPT_TEMPLATE)

In [29]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    condense_question_prompt=CONDENSE_QUESTION_PROMPT, # Custom prompt for condensing
    combine_docs_chain_kwargs={"prompt": QA_PROMPT},   # Custom prompt for QA
    return_source_documents=True, # Optional: to see which documents were retrieved
    verbose=True # Set to True for debugging to see intermediate steps
)

In [ ]:
print("\n--- Knee Osteoarthritis Chatbot ---")
print("Ask me anything about Knee Osteoarthritis based on the provided documents.")
print("Type 'quit', 'exit', or 'bye' to end the conversation.")

while True:
    try:
        user_query = input("\nYou: ")
        if user_query.lower() in ["quit", "exit", "bye"]:
            print("Bot: Goodbye! Stay healthy.")
            break
        if not user_query.strip():
            continue

        # Add a thinking indicator (optional)
        print("Bot: Thinking...")

        # Invoke the chain. It expects a dictionary with the "question".
        # The chat_history is automatically pulled from memory.
        result = qa_chain({"question": user_query})
        answer = result["answer"]

        print(f"Bot: {answer}")

        # Optional: Print source documents for debugging or transparency
        print("\n--- Sources ---")
        for doc in result.get("source_documents", []):
            print(f"- {doc.metadata.get('source', 'Unknown')}, content snippet: {doc.page_content[:100]}...")

    except Exception as e:
        print(f"An error occurred: {e}")
        # Optionally, you might want to break or offer to reset
        # For now, we'll just let the loop continue


--- Knee Osteoarthritis Chatbot ---
Ask me anything about Knee Osteoarthritis based on the provided documents.
Type 'quit', 'exit', or 'bye' to end the conversation.


Documents 1 : 
 While age is a major risk factor for osteoarthritis of the knee, young people can get it, too. For some people, it may be hereditary. For others, osteoarthritis of the knee can result from injury or infection or even from being overweight. Here are answers to your questions about knee osteoarthritis, including how it's treated and what you can do at home to ease the pain.
What Is Osteoarthritis?

Osteoarthritis, commonly known as wear-and-tear arthritis, is a condition in which the natural cushioning between joints – cartilage – wears away. When this happens, the bones of the joints rub more closely against one another with less of the shock-absorbing benefits of cartilage. The rubbing results in pain, swelling, stiffness, less ability to move, and, sometimes, the formation of bone spurs.
Who Gets Osteoarthritis of the Knee?

Documents 2 : 
 Related:
    What Are the Symptoms of Osteoarthritis?

When Is Surgery Used to Treat Knee Osteoarthritis?

If your doctor wants to

'<think>\nOkay, so I need to figure out how to write a response about knee osteoarthritis using only the provided documents. Let\'s see what each document says.\n\nThe first document defines knee osteoarthritis as wear-and-tear arthritis where cartilage wears away, causing pain, swelling, stiffness, and sometimes bone spurs. It mentions that age is a major risk factor but also notes it can occur in young people or due to injury, infection, or being overweight.\n\nNext, the document on "What Is Osteoarthritis?" elaborates that this condition occurs when cartilage between bones wears down over time, leading to joint pain and stiffness. It explains how this happens through repeated stress or certain conditions like OA itself.\n\nThen there\'s information about who gets knee osteoarthritis of the knee: it\'s common in older adults but can also affect younger people, especially those with a family history of the condition.\n\nThe document on when surgery is used mentions that if surgery is 

"Okay, let's break down knee osteoarthritis (OA) – it's a really common condition, and understanding it can help you manage it. Here's a detailed explanation:\n\n**What is Knee Osteoarthritis?**\n\nKnee osteoarthritis is a degenerative joint disease. That means it's a gradual wearing down of the cartilage in your knee joint. Cartilage is the smooth, rubbery tissue that cushions the ends of your bones and allows them to move easily together. When this cartilage breaks down, it causes pain, stiffness, and swelling in the knee.\n\n**Here's a more detailed breakdown:**\n\n1. **The Normal Knee Joint:**\n   * **Bones:** The femur (thighbone), tibia (shinbone), and patella (kneecap) all meet at the knee joint.\n   * **Cartilage:** This smooth tissue covers the ends of the bones, acting as a shock absorber and reducing friction during movement.\n   * **Synovial Fluid:** This fluid lubricates the joint, further reducing friction.\n   * **Ligaments & Muscles:** These provide stability to the joi

'Knee-osteoarthritis is a condition in which the natural cushioning between joints – cartilage – wears away, causing pain, swelling, stiffness, less mobility, and sometimes the formation of bone spurs.'